In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image

In [2]:
def genData(mean, cov, size, alpha, beta, gamma):
    X = np.random.multivariate_normal(mean, cov, size)
    X = np.c_[X, np.ones((X.shape[0], 1))]

    alp = np.mat([[1, 0, 0, 0], 
                  [0, math.cos(alpha), math.sin(alpha), 0],
                  [0, -math.sin(alpha), math.cos(alpha), 0], 
                  [0, 0, 0, 1]])
    bet = np.mat([[math.cos(beta), 0, -math.sin(beta), 0], 
                  [0, 1, 0, 0],
                  [math.sin(beta), 0, math.cos(beta), 0], 
                  [0, 0, 0, 1]])
    gam = np.mat([[math.cos(gamma), math.sin(gamma), 0, 0],
                  [-math.sin(gamma), math.cos(gamma), 0, 0], 
                  [0, 0, 1, 0], 
                  [0, 0, 0, 1]])

    alphaX = np.dot(X, alp)
    betaX = np.dot(alphaX, bet)
    gammaX = np.dot(betaX, gam)
    data = np.mat(gammaX[:, 0:3]).T
    ori = np.mat(X[:, 0:3]).T
    return data, ori

In [3]:
def PCA(data, k):
    data = data.astype('int64')
    average = np.average(data,axis=1)
#     cov = np.cov(data)
#     data = data - average
    cov = np.dot(data,data.T)/data.shape[0]
    U, Sigma, VT = np.linalg.svd(cov)
#     print(cov)
#     print(U)
#     print(Sigma)
#     print(VT)
    U = U[:, :k]
#     data = data + average
#     print(U.shape)
    newData = np.dot(U.T, data)
#     newData = newData + average
    oldData = np.dot(U, newData).T
#     oldData = oldData + average
    return U, newData, oldData.T


def PSNR(I, K):
    m, n = I.shape
    MAX = 255
    MSE = np.average(np.power(I - K,2))
    PSNR = 10 * math.log10(MAX**2 / MSE)
    return PSNR

In [4]:
mean = [0, 0, 0]
cov = [[1, 0, 0], [0, 8, 0], [0, 0, 10]]
size = 500
k = 2
n = 3
alpha = 0.15 * math.pi
beta = 0.25 * math.pi
gamma = 0.35 * math.pi
data, ori = genData(mean, cov, size, alpha, beta, gamma)
U, newData, oldData = PCA(data, k)

In [5]:
%matplotlib qt5

A = ori.T
B = data.T
C = newData
D = oldData

fig = plt.figure()
ax = plt.subplot(111, projection='3d')  # 创建一个三维的绘图工程
ax.scatter(A[:, 0].tolist(), A[:, 1].tolist(), A[:, 2].tolist(),
           marker='x')  # 绘制数据点
plt.title('original')
plt.show()

fig = plt.figure()
ax = plt.subplot(111, projection='3d')  # 创建一个三维的绘图工程
ax.scatter(B[:, 0].tolist(), B[:, 1].tolist(), B[:, 2].tolist(),
           marker='x')  # 绘制数据点
plt.title('rotational')
plt.show()

plt.figure()
plt.plot(C[0, :], C[1, :], '.', color='b')
plt.title('Dimension reduction')
plt.show()

fig = plt.figure()
ax = plt.subplot(111, projection='3d')  # 创建一个三维的绘图工程
ax.scatter(D[0, :].tolist(), D[1, :].tolist(), D[2, :].tolist(),
           marker='x')  # 绘制数据点
plt.title('reduction')
plt.show()

In [11]:
k = 8
for i in range(60):
    infile = "GSI/" + str(i) + ".jpg"
    outfile = "generalization/" + str(i) + ".jpg"
    inimg = Image.open(infile)
    data = np.mat(inimg).astype('int64')
    #     averagee = np.average(data)
    #     data = data - average
    U, newData, oldData = PCA(data, k)
    #     pcaData = pcaData + average
    outimg = Image.fromarray(oldData.astype('uint8'))
    outimg.save(outfile)
    print(i,"PSNR:",PSNR(data,oldData))
#     if oldData.any()<0 or oldData.any()>255:
#         print(False)
#     if oldData.any()<=1:
#         print(i)

0 PSNR: 17.19520658803271
1 PSNR: 17.033237637197043
2 PSNR: 19.437721012313755
3 PSNR: 19.19534838008266
4 PSNR: 18.02256351678007
5 PSNR: 16.69674612755293
6 PSNR: 21.365244557378826
7 PSNR: 18.911871944265982
8 PSNR: 19.334393525154557
9 PSNR: 17.885521658909013
10 PSNR: 20.566199611341695
11 PSNR: 17.656538272331865
12 PSNR: 18.425720476854888
13 PSNR: 20.178655288843345
14 PSNR: 20.218717321066862
15 PSNR: 17.508008530049516
16 PSNR: 16.84442251796764
17 PSNR: 18.788732207163974
18 PSNR: 16.962721593495836
19 PSNR: 21.09526579028436
20 PSNR: 17.10500856653517
21 PSNR: 20.90405690177782
22 PSNR: 20.37641387023347
23 PSNR: 17.43844393931792
24 PSNR: 21.52292959491609
25 PSNR: 20.82872844934824
26 PSNR: 17.476210318692303
27 PSNR: 18.113029016894636
28 PSNR: 18.39925442093857
29 PSNR: 18.552623194794332
30 PSNR: 19.254625587664595
31 PSNR: 19.894325595940046
32 PSNR: 17.31103885346053
33 PSNR: 17.131311692684218
34 PSNR: 18.749160121088277
35 PSNR: 16.311487585699
36 PSNR: 26.2120779